In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from geopy import distance

In [36]:
base_path='../../../30data/db_regio_bus/data'

ond_df=pd.read_csv(base_path+'/modified/on_demand_travel_mod_wdw_feat.csv',sep=',',index_col=[0])

bus_stops_df=pd.read_csv(base_path+'/bus_stops.csv',sep=',',index_col=[0])

reg_rout_df=pd.read_csv(base_path+'/regular_route_definitions.csv',sep=',',index_col=[0])

# Holidays

In [37]:
from datetime import date
import holidays

by_holidays = holidays.country_holidays('DE', subdiv='BY')

In [38]:
def add_holidays(row):
    if row['date'] in by_holidays:
        row['holiday']=1
    return row

In [39]:
ond_df['holiday']=0
ond_df=ond_df.apply(add_holidays,axis=1)

In [40]:
ond_df.head()

,date,EZone,hour,Passengers,class,"Brand, Ort","Bärnau, Marktplatz","Deggendorf, Busbahnhof","Deggendorf, Hauptbahnhof","Deggendorf, Stadtplatz",...,"Viechtach, Bahnhof/ZOB","Viechtach, Schmidstraße/Edeka","Viechtach, Stadtplatz","Vilshofen, Bahnhof","Vilshofen, Stadtplatz","Waldsassen, J.-Wiesneth-Str.","Waldsassen, Schützenstraße","Wiesau, Bf ANKUNFT",wdw_feature,holiday
0,2019-01-01 00:00:00,"Brand, Ort",0,0.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2019-01-01 01:00:00,"Brand, Ort",1,0.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2019-01-01 02:00:00,"Brand, Ort",2,0.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2019-01-01 03:00:00,"Brand, Ort",3,0.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2019-01-01 04:00:00,"Brand, Ort",4,0.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Bus Stops

In [6]:
def cutout_busstop_no(row):
    row['Nummer']=int(row['EZone'].split('-')[0])
    return row

In [7]:
ond_df['Nummer']=0
ond_df=ond_df.apply(cutout_busstop_no,axis=1)

In [8]:
bus_stops_df.head()

,name_sanitized,long,lat,Name,Kürzel
Nummer,,,,,
4972,"Fürsteneck, Abzw Bf",13.470781,48.717882,"Fürsteneck, Abzw Bf",PFAB
4792,"Fürsteneck, Metzgerei Stegbauer",13.461119,48.720970,"Fürsteneck, Metzgerei Stegbauer",4792
4320,Voglöd bei Prag,13.472936,48.714230,Voglöd bei Prag,PVOG
4887,"Guttenhofen, Abzw",13.527810,48.714984,"Guttenhofen, Abzw",PAGH
4429,"Künzing, Römerbad Quintana",13.095707,48.667151,"Künzing, Römerbad Quintana",4429


In [9]:
bus_stops_df[bus_stops_df.index==15964]

,name_sanitized,long,lat,Name,Kürzel
Nummer,,,,,
15964,"Salzweg, Außenstelle LRA",13.483832,48.604788,"Salzweg, Außenstelle LRA",15964


In [10]:
ond_df_bus_stops=pd.unique(ond_df['Nummer']).tolist()
bus_stops=pd.unique(bus_stops_df.index).tolist()

In [11]:
len(ond_df_bus_stops)

50

In [12]:
len(bus_stops)

1309

In [13]:
bus_stops_reduced_df=bus_stops_df.copy()

for i in bus_stops:
    if i not in ond_df_bus_stops:
        bus_stops_reduced_df.drop(i,inplace=True)

bus_stops_reduced_df.reset_index(inplace=True)

In [14]:
len(bus_stops_reduced_df)

50

In [15]:
bus_stops_reduced_df.head()

,Nummer,name_sanitized,long,lat,Name,Kürzel
0,4563,"Tittling, Marktplatz",13.380432,48.728014,"Tittling, Marktplatz",PATM
1,4614,"Neukirchen v.W., Rathaus",13.382881,48.686785,"Neukirchen v.W., Rathaus",PNEU
2,5425,"Breitenberg, Passauer-Str.",13.793847,48.703004,"Breitenberg, Passauer-Str.",PABR
3,4883,"Hutthurm, Bayerwaldstraße",13.474553,48.673251,"Hutthurm, Bayerwaldstraße",PDHU
4,4809,Kreuzstraße Tankstelle,13.483620,48.669139,Kreuzstraße Tankstelle,PKTS


# Add long and lat as diff from geograpic center

In [16]:
geo_distances={
    'name_sanitized':[],
    'geo_distance':[]
}

for i in range(len(bus_stops_reduced_df)):
    row=bus_stops_reduced_df.iloc[i]
    geo_distances['name_sanitized'].append(row['name_sanitized'])

    current_loc=(row['lat'],row['long'])
    summed_dist=0
    for i2 in range(len(bus_stops_reduced_df)):
        other_row=bus_stops_reduced_df.iloc[i2]

        summed_dist+=distance.distance(current_loc,(other_row['lat'],other_row['long'])).km

    geo_distances['geo_distance'].append(summed_dist/len(bus_stops_reduced_df))


geo_distances=pd.DataFrame(data=geo_distances)

In [17]:
geo_distances['geo_distance']=geo_distances['geo_distance']-geo_distances.sort_values('geo_distance').iloc[0]['geo_distance']

In [18]:
bus_stops_reduced_df=pd.merge(bus_stops_reduced_df,geo_distances,on=['name_sanitized'])

In [19]:
bus_stops_reduced_df.head()

,Nummer,name_sanitized,long,lat,Name,Kürzel,geo_distance
0,4563,"Tittling, Marktplatz",13.380432,48.728014,"Tittling, Marktplatz",PATM,8.652935
1,4614,"Neukirchen v.W., Rathaus",13.382881,48.686785,"Neukirchen v.W., Rathaus",PNEU,5.366303
2,5425,"Breitenberg, Passauer-Str.",13.793847,48.703004,"Breitenberg, Passauer-Str.",PABR,18.811163
3,4883,"Hutthurm, Bayerwaldstraße",13.474553,48.673251,"Hutthurm, Bayerwaldstraße",PDHU,3.410099
4,4809,Kreuzstraße Tankstelle,13.483620,48.669139,Kreuzstraße Tankstelle,PKTS,3.170867


# Ad long and alt as diff from passangers center

In [20]:
ond_df_pass=ond_df.groupby(['Nummer'])[['Passengers']].sum()

In [21]:
pass_center=bus_stops_reduced_df[bus_stops_reduced_df['Nummer']==ond_df_pass.sort_values('Passengers', ascending=False).head(1).index[0]]
pass_center_coor=(pass_center['lat'].values[0],pass_center['long'].values[0])

In [22]:
def set_distance(row):
    row['pass_distance']=distance.distance(pass_center_coor,(row['lat'],row['long'])).km
    return row

In [23]:
bus_stops_reduced_df['pass_distance']=0
bus_stops_reduced_df=bus_stops_reduced_df.apply(set_distance,axis=1)

In [24]:
bus_stops_reduced_df.head()

,Nummer,name_sanitized,long,lat,Name,Kürzel,geo_distance,pass_distance
0,4563,"Tittling, Marktplatz",13.380432,48.728014,"Tittling, Marktplatz",PATM,8.652935,17.880484
1,4614,"Neukirchen v.W., Rathaus",13.382881,48.686785,"Neukirchen v.W., Rathaus",PNEU,5.366303,13.496562
2,5425,"Breitenberg, Passauer-Str.",13.793847,48.703004,"Breitenberg, Passauer-Str.",PABR,18.811163,28.995543
3,4883,"Hutthurm, Bayerwaldstraße",13.474553,48.673251,"Hutthurm, Bayerwaldstraße",PDHU,3.410099,11.131113
4,4809,Kreuzstraße Tankstelle,13.483620,48.669139,Kreuzstraße Tankstelle,PKTS,3.170867,10.805053


In [25]:
bus_stops_reduced_df.sort_values('geo_distance').head(10)

,Nummer,name_sanitized,long,lat,Name,Kürzel,geo_distance,pass_distance
25,4028,"Passau, Nagelschmiedgasse",13.461146,48.576102,"Passau, Nagelschmiedgasse",4028,0.000000,0.726527
23,4030,"Passau, Am Schanzlturm",13.455990,48.575090,"Passau, Am Schanzlturm",PAST,0.007678,0.329789
22,4010,"Passau, Am Schanzl(Busbucht)",13.456125,48.574415,"Passau, Am Schanzl(Busbucht)",PAZB,0.017493,0.328313
24,4000,"Passau, Hbf",13.451679,48.574309,"Passau, Hbf",PASB,0.048324,0.000000
26,4029,"Passau, Rathaus",13.468362,48.575478,"Passau, Rathaus",PPRH,0.056710,1.237980
29,4001,"Passau, ZOB",13.455872,48.572521,"Passau, ZOB",PKEP,0.061279,0.367815
27,9750,"Passau, Reisebüro Niedermayer",13.450428,48.574233,"Passau, Reisebüro Niedermayer",PPRN,0.063497,0.092683
28,4016,"Passau, Römerplatz",13.470745,48.574908,"Passau, Römerplatz",PARP,0.093521,1.408568
18,15964,"Salzweg, Außenstelle LRA",13.483832,48.604788,"Salzweg, Außenstelle LRA",15964,0.229193,4.136960
17,4807,"Angl, Bäckerei Vogl",13.482593,48.609015,"Angl, Bäckerei Vogl",4807,0.272325,4.482753


In [26]:
bus_stops_reduced_df.sort_values('pass_distance').head(10)

,Nummer,name_sanitized,long,lat,Name,Kürzel,geo_distance,pass_distance
24,4000,"Passau, Hbf",13.451679,48.574309,"Passau, Hbf",PASB,0.048324,0.000000
27,9750,"Passau, Reisebüro Niedermayer",13.450428,48.574233,"Passau, Reisebüro Niedermayer",PPRN,0.063497,0.092683
22,4010,"Passau, Am Schanzl(Busbucht)",13.456125,48.574415,"Passau, Am Schanzl(Busbucht)",PAZB,0.017493,0.328313
23,4030,"Passau, Am Schanzlturm",13.455990,48.575090,"Passau, Am Schanzlturm",PAST,0.007678,0.329789
29,4001,"Passau, ZOB",13.455872,48.572521,"Passau, ZOB",PKEP,0.061279,0.367815
25,4028,"Passau, Nagelschmiedgasse",13.461146,48.576102,"Passau, Nagelschmiedgasse",4028,0.000000,0.726527
26,4029,"Passau, Rathaus",13.468362,48.575478,"Passau, Rathaus",PPRH,0.056710,1.237980
28,4016,"Passau, Römerplatz",13.470745,48.574908,"Passau, Römerplatz",PARP,0.093521,1.408568
33,5381,"Passau, Lindau",13.501416,48.585161,"Passau, Lindau",5381,0.533987,3.863264
18,15964,"Salzweg, Außenstelle LRA",13.483832,48.604788,"Salzweg, Außenstelle LRA",15964,0.229193,4.136960


In [27]:
ond_df_final=pd.merge(ond_df,bus_stops_reduced_df[['long','lat','geo_distance','pass_distance','Nummer']],on='Nummer')

In [42]:
ond_df.to_csv('./on_demand_travel_mod_wdw_busstops_feat.csv')